In [419]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [420]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DST = 4
K = 5

In [421]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Bye'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

In [422]:
tupDef = [('', 'Bye'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

In [423]:
tupKicker = [('', 'Bye'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [424]:
#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Position'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [425]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [426]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DST, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(K, pd.io.html.read_html(url_kicker_season17)[0]) 

In [427]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [428]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [429]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [430]:
def setDataFrame(dframe, dataType, index, fixXa0):
    nameIndex = []
    dataMatrix = []
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    if(fixXa0):
        nameIndex = nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    return(dframe)



In [575]:
def setDataFrame_weeks(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos']
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    
    #Fix the names for QB, WR, TE, RB
    if(position == ('QB' or 'RB' or 'TE' or 'WR' )):
        print('IN HERE!!!')
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    elif(position == ('DST' or 'K')):
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            #nameIndex = frameArray[dataType].get('Team')
            nameIndex = dframe.get('Team')
            print(dframe)
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        retframe = (dframe.rename(d1, axis = 'index'))
        retframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    #Create the dataframe for the keys using keyList argument 
    d2 = {('','Pos'): keyList['Pos'], 
          ('', 'PPR'): keyList['StdScore'],
          ('', 'Week'): keyList['Week'],
          ('', 'Year'): keyList['Year']
         }
    
    kframe = DataFrame(d2, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        

    return(retframe)


In [570]:
#MAIN STARTS HERE
frameArray = []
loadWebData(frameArray)


In [576]:
#Create multi-index from tuple
index = pd.MultiIndex.from_tuples(tupOffense)
frameQB = setDataFrame(frameArray[QB], QB, index, fixXa0 = True)
frameRB = setDataFrame(frameArray[RB], RB, index, fixXa0 = True)
frameWR = setDataFrame(frameArray[WR], WR, index, fixXa0 = True)
frameTE = setDataFrame(frameArray[TE], TE, index, fixXa0 = True)

#index = pd.MultiIndex.from_tuples(tupDef)
#frameDef = setDataFrame(frameArray[DST], DST, index, fixXa0 = False)

#TODO - CREATE BOTTOM CODE WORKS. MAKE IT CLEAN! 
frameQB = setDataFrame_weeks(tframeArray[6][0], tframeArray[6][1] , index)
#frameK  = setDataFrame_weeks(tframeArray[4][0], tframeArray[4][1], index)

#index = pd.MultiIndex.from_tuples(tupKicker)
#frameKicker = setDataFrame(frameArray[K], K, index, fixXa0 = False)



IN HERE!!!
                                   
                 PPR Pos Week  Year
Matt Schaub        1  QB    2  2010
Aaron Rodgers      1  QB    2  2010
Jay Cutler         1  QB    2  2010
Mark Sanchez       1  QB    2  2010
Peyton Manning     1  QB    2  2010
Philip Rivers      1  QB    2  2010
Michael Vick       1  QB    2  2010
Matt Ryan          1  QB    2  2010
Kyle Orton         1  QB    2  2010
Shaun Hill         1  QB    2  2010
Donovan McNabb     1  QB    2  2010
Josh Freeman       1  QB    2  2010
Drew Brees         1  QB    2  2010
Alex Smith         1  QB    2  2010
Matt Hasselbeck    1  QB    2  2010
Tony Romo          1  QB    2  2010
Tom Brady          1  QB    2  2010
Sam Bradford       1  QB    2  2010
Eli Manning        1  QB    2  2010
Seneca Wallace     1  QB    2  2010
Chad Henne         1  QB    2  2010
Bruce Gradkowski   1  QB    2  2010
Kerry Collins      1  QB    2  2010
Matt Moore         1  QB    2  2010
Carson Palmer      1  QB    2  2010
Luke McCown      

In [577]:
frameQB

Passing                     Rushing      ...   \
                      Bye   Pts     Att Cmp  Yds TD Int 2Pt     Att Yds  ...    
Matt Schaub       HOU@WAS  35.0      52  38  497  3   1   0       1   2  ...    
Aaron Rodgers      BUF@GB  30.0      29  19  255  2   0   0       5  20  ...    
Jay Cutler        CHI@DAL  29.0      29  21  277  3   0   0       5   3  ...    
Mark Sanchez       NE@NYJ  28.0      30  21  220  3   0   1       4   2  ...    
Peyton Manning    NYG@IND  28.0      26  20  255  3   0   0       3  -3  ...    
Philip Rivers      JAX@SD  27.0      29  22  334  3   2   0       0   0  ...    
Michael Vick      PHI@DET  26.0      34  21  284  2   0   0       8  37  ...    
Matt Ryan         ARI@ATL  26.0      32  21  225  3   0   0       6   6  ...    
Kyle Orton        SEA@DEN  24.0      35  25  307  2   0   0       3  -5  ...    
Shaun Hill        PHI@DET  24.0      45  25  335  2   2   1       3  16  ...    
Donovan McNabb    HOU@WAS  23.0      38  28  426  1   0   0       0   0  ...    
Josh Freeman       TB@CAR  23.0      24  12  178  2   0   0       4  43  ...    
Drew Brees          NO@SF  22.0      38  28  254  2   0   0       0   0  ...    
Alex Smith          NO@SF  17.0      32  23  275  1   2   1       4  28  ...    
Matt Hasselbeck   SEA@DEN  17.0      35  20  233  1   3   0       3  21  ...    
Tony Romo         CHI@DAL  16.0      51  34  374  1   2   0       1  -1  ...    
Tom Brady          NE@NYJ  16.0      36  20  248  2   2   0       0   0  ...    
Sam Bradford      STL@OAK  16.0      25  14  167  2   1   0       0   0  ...    
Eli Manning       NYG@IND  14.0      24  13  161  2   1   0       1   6  ...    
Seneca Wallace     KC@CLE  13.0      31  16  229  1   1   0       1   4  ...    
Chad Henne        MIA@MIN  10.0      15   9  114  1   0   0       2   0  ...    
Bruce Gradkowski  STL@OAK  10.0      21  11  162  1   1   0       4   0  ...    
Kerry Collins     PIT@TEN  10.0      25  17  149  1   1   1       2  -1  ...    
Matt Moore         TB@CAR   8.0      16   6  125  1   1   0       1   2  ...    
Carson Palmer     BAL@CIN   6.0      35  16  167  0   0   0       3  -1  ...    
Luke McCown        JAX@SD   4.0      19  11  120  0   0   0       1   4  ...    
David Garrard      JAX@SD   4.0      23  15  173  1   4   0       1  -1  ...    
Joe Flacco        BAL@CIN   4.0      39  17  154  1   4   0       3   9  ...    
Matt Cassel        KC@CLE   3.0      28  16  176  0   2   0       4   5  ...    
Brett Favre       MIA@MIN   2.0      36  22  225  0   3   0       0   0  ...    
Jason Campbell    STL@OAK   2.0      15   8   87  0   1   0       3  19  ...    
Derek Anderson    ARI@ATL   2.0      31  17  161  0   2   0       0   0  ...    
Trent Edwards      BUF@GB   1.0      18  11  102  0   2   0       3  12  ...    
Dennis Dixon      PIT@TEN   1.0       6   4   18  0   0   0       3  28  ...    
Charlie Batch     PIT@TEN   1.0      11   5   25  0   0   0       1   0  ...    
Billy Volek        JAX@SD   0.0       0   0    0  0   0   0       1  -1  ...    
Jimmy Clausen      TB@CAR   0.0      13   7   59  0   1   0       1   0  ...    
Vince Young       PIT@TEN  -2.0      10   7   66  0   2   0       2  12  ...    
Max Hall          ARI@ATL  -2.0       2   1    3  0   1   0       0   0  ...    

                 Receiving            Fumbles                        
                       Rec Yds TD 2Pt      FL TD PPR Pos Week  Year  
Matt Schaub              0   0  0   0       0  0   1  QB    2  2010  
Aaron Rodgers            0   0  0   0       0  0   1  QB    2  2010  
Jay Cutler               0   0  0   0       0  0   1  QB    2  2010  
Mark Sanchez             0   0  0   0       0  0   1  QB    2  2010  
Peyton Manning           0   0  0   0       0  0   1  QB    2  2010  
Philip Rivers            0   0  0   0       0  0   1  QB    2  2010  
Michael Vick             0   0  0   0       0  0   1  QB    2  2010  
Matt Ryan                0   0  0   0       1  0   1  QB    2  2010  
Kyle Orton

In [435]:
columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)

In [436]:
frame = [frameMaster, frameKicker, frameQB]
frameMaster = pd.concat(frame)
frameMaster


\
                     Bye Game  Opp  PPR Position    Pts Team Week Year   
Greg Zuerlein        8.0  NaN  NaN  NaN      NaN  170.0  NaN  NaN  NaN   
Stephen Gostkowski   9.0  NaN  NaN  NaN      NaN  164.0  NaN  NaN  NaN   
Robbie Gould        11.0  NaN  NaN  NaN      NaN  153.0  NaN  NaN  NaN   
Matt Bryant          0.0  NaN  NaN  NaN      NaN  153.0  NaN  NaN  NaN   
Justin Tucker       10.0  NaN  NaN  NaN      NaN  151.0  NaN  NaN  NaN   
Harrison Butker     11.0  NaN  NaN  NaN      NaN  150.0  NaN  NaN  NaN   
Chris Boswell        9.0  NaN  NaN  NaN      NaN  150.0  NaN  NaN  NaN   
Wil Lutz             0.0  NaN  NaN  NaN      NaN  148.0  NaN  NaN  NaN   
Matt Prater          7.0  NaN  NaN  NaN      NaN  144.0  NaN  NaN  NaN   
Kai Forbath          9.0  NaN  NaN  NaN      NaN  142.0  NaN  NaN  NaN   
Ryan Succop          8.0  NaN  NaN  NaN      NaN  140.0  NaN  NaN  NaN   
Stephen Hauschka     6.0  NaN  NaN  NaN      NaN  130.0  NaN  NaN  NaN   
Jake Elliott        10.0  NaN  NaN  NaN      NaN  127.0  NaN  NaN  NaN   
Phil Dawson          8.0  NaN  NaN  NaN      NaN  127.0  NaN  NaN  NaN   
Graham Gano         11.0  NaN  NaN  NaN      NaN  121.0  NaN  NaN  NaN   
Adam Vinatieri      11.0  NaN  NaN  NaN      NaN  119.0  NaN  NaN  NaN   
Chandler Catanzaro  11.0  NaN  NaN  NaN      NaN  108.0  NaN  NaN  NaN   
Brandon McManus      0.0  NaN  NaN  NaN      NaN  105.0  NaN  NaN  NaN   
Blair Walsh          6.0  NaN  NaN  NaN      NaN  100.0  NaN  NaN  NaN   
Ka'imi Fairbairn     7.0  NaN  NaN  NaN      NaN   98.0  NaN  NaN  NaN   
Cody Parkey          1.0  NaN  NaN  NaN      NaN   91.0  NaN  NaN  NaN   
Giorgio Tavecchio   10.0  NaN  NaN  NaN      NaN   87.0  NaN  NaN  NaN   
Randy Bullock        6.0  NaN  NaN  NaN      NaN   87.0  NaN  NaN  NaN   
Josh Lambo           8.0  NaN  NaN  NaN      NaN   83.0  NaN  NaN  NaN   
Patrick Murray       1.0  NaN  NaN  NaN      NaN   82.0  NaN  NaN  NaN   
Aldrick Rosas        8.0  NaN  NaN  NaN      NaN   80.0  NaN  NaN  NaN   
Mason Crosby         8.0  NaN  NaN  NaN      NaN   80.0  NaN  NaN  NaN   
Dan Bailey           6.0  NaN  NaN  NaN      NaN   77.0  NaN  NaN  NaN   
Zane Gonzalez        9.0  NaN  NaN  NaN      NaN   74.0  NaN  NaN  NaN   
Dustin Hopkins       0.0  NaN  NaN  NaN      NaN   60.0  NaN  NaN  NaN   
...                  ...  ...  ...  ...      ...    ...  ...  ...  ...   
T.J. Yates           7.0  NaN  NaN  NaN      NaN   41.0  NaN  NaN  NaN   
Kevin Hogan          9.0  NaN  NaN  NaN      NaN   38.0  NaN  NaN  NaN   
Sam Bradford         9.0  NaN  NaN  NaN      NaN   32.0  NaN  NaN  NaN   
Bryce Petty         11.0  NaN  NaN  NaN      NaN   24.0  NaN  NaN  NaN   
David Fales          1.0  NaN  NaN  NaN      NaN   20.0  NaN  NaN  NaN   
Paxton Lynch         0.0  NaN  NaN  NaN      NaN   18.0  NaN  NaN  NaN   
E.J. Manuel         10.0  NaN  NaN  NaN      NaN   15.0  NaN  NaN  NaN   
Geno Smith           8.0  NaN  NaN  NaN      NaN   13.0  NaN  NaN  NaN   
Ryan Mallett        10.0  NaN  NaN  NaN      NaN   13.0  NaN  NaN  NaN   
Nathan Peterman      6.0  NaN  NaN  NaN      NaN   12.0  NaN  NaN  NaN   
Landry Jones         9.0  NaN  NaN  NaN      NaN   12.0  NaN  NaN  NaN   
Patrick Mahomes     10.0  NaN  NaN  NaN      NaN   10.0  NaN  NaN  NaN   
Nate Sudfeld        10.0  NaN  NaN  NaN      NaN    7.0  NaN  NaN  NaN   
Sean Mannion         8.0  NaN  NaN  NaN      NaN    5.0  NaN  NaN  NaN   
Matt Cassel          8.0  NaN  NaN  NaN      NaN    5.0  NaN  NaN  NaN   
Joe Webb             6.0  NaN  NaN  NaN      NaN    3.0  NaN  NaN  NaN   
Cody Kessler         9.0  NaN  NaN  NaN      NaN    2.0  NaN  NaN  NaN   
Scott Tolzien       11.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
Cooper Rush          6.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
A.J. McCarron        6.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
Chad Henne           8.0  NaN  NaN  NaN      NaN    0.0  NaN  NaN  NaN   
Taylor Heinicke      7.0  NaN  NaN  NaN      NaN    0.0  NaN  NaN  NaN   
Au

# TEST SECTION STARTS AFTER THIS LINE 

In [438]:
def setDataFrame_weeks_scratch(dframe, keyList, dataType, index, fixXa0):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    
    #Fix the names for QB, WR, TE, RB
    if(fixXa0):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    #Create the dataframe for the keys using keyList argument 
    d2 = {('','Pos'): keyList['Pos'], 
          ('', 'PPR'): keyList['StdScore'],
          ('', 'Week'): keyList['Week'],
          ('', 'Year'): keyList['Year']
         }
    
    kframe = DataFrame(d2, nameIndex)
    dframe = pd.concat([dframe, kframe], axis=1, join_axes=[dframe.index])    
        

    return(dframe)


In [337]:
tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

In [399]:
tframeArray = []
file = open('testpickle', 'rb')
tframeArray = pickle.load(file)
file.close()

In [501]:
tframeArray[6][0]
tframeArray[4][1]['Pos']

'K'

In [547]:
pd.concat([frameA, frameB], axis = 1, join_axes = [frameA.index])

blah               
               Pos Week Score bye
Tom Brady       QB    2   100   2
Russel Wilson   QB    2   100   2

In [548]:
nameindex = ['Tom Brady', 'Russel Wilson']
t1 = {('blah', 'Pos'): 'QB', ('blah', 'Week'):2}
t2 = {('blah', 'Score'): 100, ('blah', 'bye'):2}
frameA = DataFrame(t1, index=nameindex)

In [549]:
frameA = DataFrame(t1, index=nameindex)
frameB = DataFrame(t2, index=nameindex)

In [554]:
frameA = pd.concat([frameA, frameB], axis = 1, join_axes = [frameA.index])

In [559]:
t3 = {('new', 'yds'): 100}
frameC = DataFrame(t3, index=nameindex)
pd.concat([frameA, frameC], axis = 1, join_axes = [frameA.index])

blah                 new
               Pos Week Score bye  yds
Tom Brady       QB    2   100   2  100
Russel Wilson   QB    2   100   2  100

'QB'

In [301]:
{('blah', 'Pos'): tframeArray[0][1]['Pos']}

{('blah', 'Pos'): 'QB'}

In [563]:
tframeArray[1][0]

Passing Rushing Receiving  \
                             Player     Game  Pts*     Att     Cmp       Yds   
0             Arian FosterA. Foster  IND@HOU  41.0       0       0         0   
1                Matt ForteM. Forte  DET@CHI  31.0       0       0         0   
2           Chris JohnsonC. Johnson  OAK@TEN  26.0       0       0         0   
3        Darren McFaddenD. McFadden  OAK@TEN  20.0       0       0         0   
4   Rashard MendenhallR. Mendenhall  ATL@PIT  19.0       0       0         0   
5          Jamaal CharlesJ. Charles    SD@KC  15.0       0       0         0   
6                Jahvid BestJ. Best  DET@CHI  15.0       0       0         0   
7            Pierre ThomasP. Thomas   MIN@NO  14.0       0       0         0   
8              Ronnie BrownR. Brown  MIA@BUF  14.0       0       0         0   
9              LeSean McCoyL. McCoy   GB@PHI  13.0       0       0         0   
10        Tim HightowerT. Hightower  ARI@STL  13.0       0       0         0   
11        Ahmad BradshawA. Bradshaw  CAR@NYG  13.0       0       0         0   
12         Knowshon MorenoK. Moreno  DEN@JAX  12.0       0       0         0   
13           Peyton HillisP. Hillis   CLE@TB  11.0       0       0         0   
14  Maurice Jones-DrewM. Jones-Drew  DEN@JAX  10.0       0       0         0   
15           Cedric BensonC. Benson   CIN@NE  10.0       0       0         0   
16      Carnell WilliamsC. Williams   CLE@TB   9.0       0       0         0   
17            Javon RingerJ. Ringer  OAK@TEN   9.0       0       0         0   
18       Adrian PetersonA. Peterson   MIN@NO   9.0       0       0         0   
19         Steven JacksonS. Jackson  ARI@STL   8.0       0       0         0   
20  LaDainian TomlinsonL. Tomlinson  BAL@NYJ   7.0       0       0         0   
21             Fred TaylorF. Taylor   CIN@NE   7.0       0       0         0   
22                 John KuhnJ. Kuhn   GB@PHI   7.0       0       0         0   
23        Brandon JacksonB. Jackson   GB@PHI   7.0       0       0         0   
24                Frank GoreF. Gore   SF@SEA   7.0       0       0         0   
25        Ricky WilliamsR. Williams  MIA@BUF   6.0       0       0         0   
26     DeAngelo WilliamsD. Williams  CAR@NYG   6.0       0       0         0   
27          Chester TaylorC. Taylor  DET@CHI   6.0       0       0         0   
28          Clinton PortisC. Portis  DAL@WAS   6.0       0       0         0   
29           Ryan MathewsR. Mathews    SD@KC   6.0       0       0         0   
..                              ...      ...   ...     ...     ...       ...   
60     Rock CartwrightR. Cartwright  OAK@TEN   1.0       0       0         0   
61          Leonard WeaverL. Weaver   GB@PHI   0.0       0       0         0   
62         Darren SprolesD. Sproles    SD@KC   0.0       0       0         0   
63           C.J. SpillerC. Spiller  MIA@BUF   0.0       0       0         0   
64        Jason SnellingJ. Snelling  ATL@PIT   0.0       0       0         0   
65           Mike SellersM. Sellers  DAL@WAS   0.0       0       0         0   
66     Tony RichardsonT. Richardson  BAL@NYJ   0.0       0       0         0   
67        Jerious NorwoodJ. Norwood  ATL@PIT   0.0       0       0         0   
68            Moran NorrisM. Norris   SF@SEA   0.0       0       0         0   
69            Sammy MorrisS. Morris   CIN@NE   0.0       0       0         0   
70            Mewelde MooreM. Moore  ATL@PIT   0.0       0       0         0   
71     Dexter McClusterD. McCluster    SD@KC   0.0       0       0         0   
72         Le'Ron McClainL. McClain  BAL@NYJ   0.0       0       0         0   
73             Mike KarneyM. Karney  ARI@STL   0.0       0       0         0   
74               Greg JonesG. Jones  DEN@JAX   0.0       0       0         0   
75          Larry JohnsonL. Johnson  DAL@WAS   0.0       0       0         0   
76            Jacob HesterJ. Hester    SD@KC   0.0       0       0         0   
77               Ahmard HallA. Hall  OAK@TEN   0.0 